# Drowsiness Detection using OpenCV
* With the help of Computer Vision we are trying to detect the drowsiness.
* Here we are using Haar Cascades files for object detection. 
* We also use numpy ,keras and time libraries for this project.

### Importing all the required libraries

In [1]:
import os
import cv2
from keras.models import load_model
import numpy as np
import time

Using TensorFlow backend.


### Getting the required Haar Cascades XML files 
* Here face variable stores the XML file required for face detection.
* leye stores the XML file required for left eye detection.
* reye stores the XML file required for right eye detection.

In [2]:
face=cv2.CascadeClassifier('C:\\Users\\EKTA\\Downloads\\haarcascades-20201007T081335Z-001\\haarcascades\\haarcascade_frontalface_alt.xml')
leye=cv2.CascadeClassifier('C:\\Users\\EKTA\\Downloads\\haarcascades-20201007T081335Z-001\\haarcascades\\haarcascade_lefteye_2splits.xml')
reye=cv2.CascadeClassifier('C:\\Users\\EKTA\\Downloads\\haarcascades-20201007T081335Z-001\\haarcascades\\haarcascade_righteye_2splits.xml')
lbl=['Close','Open']

#### Accessing the model file
The model variable consist of the model architecture. The model we used is built with keras using CNN(Concolutional Neural Network). CNN is a special type of neural network which performs extremely well for image classification purposes. 

In [3]:
model = load_model('C:\\Users\\EKTA\\Downloads\\Drowsiness detection\\Drowsiness detection\\models\\cnncat2.h5')
path = os.getcwd()
print(path)


C:\Users\EKTA


We store the web cam image in 'cap' variable. Then, we convert it into greyscale ,resize and normalize it.
Now we detect the face in the image and create a Region Of Interest(ROI).
Doing the same as above we detect both the eyes and create a Region Of Interest(ROI).
Now we use the CNN model for the predicting each eye.
If the predicted value is 1 that means the eye(s) is(are) open. Else the eyes closed.

In [4]:
cap=cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]
while True:
    ret,frame=cap.read()
    height,width=frame.shape[:2]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye=leye.detectMultiScale(gray)
    right_eye=reye.detectMultiScale(gray)
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,255),thickness=1)
    for(x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),5)
    for(x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye=cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye=cv2.resize(r_eye,(24,24))
        r_eye=r_eye/255
        r_eye=r_eye.reshape(24,24,-1)
        r_eye=np.expand_dims(r_eye,axis=0)
        rpred=model.predict_classes(r_eye)
        if(rpred[0]==1):
            lbl='Open'
        else:
            lbl='Closed'
        break
    for(x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye=cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
        l_eye=cv2.resize(l_eye,(24,24))
        l_eye=l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye=np.expand_dims(l_eye,axis=0)
        lpred=model.predict_classes(l_eye)
        if lpred[0]==1:
            lbl='Open'
        else:
            lbl='Closed'
    if (rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,'Closed',(10,height-20),font,1,(255,255,255),1,cv2.LINE_AA)
    else:
        score=score-1
        cv2.putText(frame,'Open',(10,height-20),font,1,(255,255,255),1,cv2.LINE_AA)
    if score<0:
        score=0
    cv2.putText(frame,'Score:'+str(score),(100,height-20),font,1,(255,255,255),1,cv2.LINE_AA)
    if(score>5):
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
        except: 
            isplaying=False
            pass
        if(thicc<16):
            thicc=thicc+2
        else:
            thicc=thicc-2
            if thicc<2:
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()